<a href="https://colab.research.google.com/github/jaimeevers/electrodynamics/blob/master/Official_07_laplace_equation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving Laplace's Equation
In this tutorial, we are going to use the finite element method to solve Laplace's equation.

## The System

The physical system under study is a conducting rectangular pipe whose four sides can be held at different potentials or left to vary freely.

## Acknowledgements

The installation commands below come from the [FEM on CoLab](https://fem-on-colab.github.io/index.html) project.

Thanks to all of the contributors to these projects!

# Install and Load Packages

These are the commands we use to install FEniCSx, Gmsh, and **multiphenicsx**.

In [ ]:
try:
    # Import gmsh library for generating meshes.
    import gmsh
except ImportError:
    # If it is not available, install it.  Then import it.
    !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
    import gmsh

--2022-10-24 03:22:47--  https://fem-on-colab.github.io/releases/gmsh-install.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2175 (2.1K) [application/x-sh]
Saving to: ‘/tmp/gmsh-install.sh’

/tmp/gmsh-install.s 100%[===================>]   2.12K  --.-KB/s    in 0s      

2022-10-24 03:22:47 (35.4 MB/s) - ‘/tmp/gmsh-install.sh’ saved [2175/2175]

+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ GMSH_INSTALLED=/usr/local/share/fem-on-colab/gmsh.installed
+ [[ ! -f /usr/local/share/fem-on-colab/gmsh.installed ]]
+ H5PY_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/ac0fe3a/releases/h5py-install.sh
+ [[ https://github.com/fem-on-colab/fem-on-colab.github.io/raw/ac0fe3a/releases/h5py-install.sh == http* ]]
+ H5PY_INSTALL_SCRIPT_DOWNLOAD=http

In [ ]:
try:
    # Import FEniCSx libraries for finite element analysis.
    import dolfinx
except ImportError:
    # If they are not found, install them.  Then import them.
    !wget "https://fem-on-colab.github.io/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

--2022-10-24 03:23:41--  https://fem-on-colab.github.io/releases/fenicsx-install-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.110.153, 185.199.111.153, 185.199.108.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3055 (3.0K) [application/x-sh]
Saving to: ‘/tmp/fenicsx-install.sh’

/tmp/fenicsx-instal 100%[===================>]   2.98K  --.-KB/s    in 0s      

2022-10-24 03:23:42 (26.9 MB/s) - ‘/tmp/fenicsx-install.sh’ saved [3055/3055]

+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FENICSX_INSTALLED=/usr/local/share/fem-on-colab/fenicsx.installed
+ [[ ! -f /usr/local/share/fem-on-colab/fenicsx.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/f0d2856/releases/pybind11-install.sh
+ [[ https://github.com/fem-on-colab/fem-on-colab.github.io/raw/f0d2856/releases/pybind11-install.sh == http* ]]


In [ ]:
try:
    # Import multiphenicsx, mainly for plotting.
    import multiphenicsx
except ImportError:
    # If they are not found, install them.
    !pip3 install "multiphenicsx@git+https://github.com/multiphenics/multiphenicsx.git@8b97b4e"
    import multiphenicsx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/multiphenics/multiphenicsx.git (to revision 8b97b4e) to /tmp/pip-install-8tvfvx2c/multiphenicsx_54dcc1b5a6694eae8a512fc2d7c0116f
  Running command git clone -q https://github.com/multiphenics/multiphenicsx.git /tmp/pip-install-8tvfvx2c/multiphenicsx_54dcc1b5a6694eae8a512fc2d7c0116f
  Running command git checkout -q 8b97b4e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for multiphenicsx: filename=multiphenicsx-0.2.dev1-py3-none-any.whl size=42586 sha256=f7a48254e73a05cd0f7ddcd67ec55b0d37b42db5b1d2ee686461022abc4c554b
  Stored in directory: /tmp/pip-ephem-wheel-cache-4jp3p18r/wheels/a4/44/a9/e578d5b67de6acd8e975bd56c96ee36944a693ee3d7bc91026
Successfully built multiphenicsx


Everything we need should be installed now!

If you "Restart runtime" from the "Runtime" menu, all of your data will be reset, but the packages will remain installed.

Let's load the packages we need, and get started!

In [ ]:
# Everything should be installed now.
# Import the rest of what we need.

import dolfinx.fem
import dolfinx.io
import gmsh
import mpi4py.MPI
import numpy as np
import petsc4py.PETSc
import ufl
import multiphenicsx.fem
import multiphenicsx.io

# Concstruct the Model

The physical system under study is a conducting rectangular pipe whose four sides can be held at different potentials or left to vary freely.

The geometric model is a rectangle, representing the cross section of the pipe.

In [ ]:
## Adjust the parameters in this cell.
# Define the shape: length and width of the rectangle
length = 2
width = 2

In [ ]:
# Create a rectangle.
# Locate the center.
x0 = 0
y0 = 0
z0 = 0

# Give input shorter nicknames.
L = length
W = width

# Locate the corner.
x0 = x0 - L/2
y0 = y0 - W/2

# Tell Gmsh how many dimensions we are using.
dim = 2

# Grid size parameter.  Make it smaller for higher resolution.
delta = 0.1

# Create a new model.
gmsh.initialize()
gmsh.model.add("mesh")

# Define points: corners of the rectangle.
p0 = gmsh.model.geo.addPoint(x0,y0, z0, delta)
p1 = gmsh.model.geo.addPoint(x0+L, y0, z0, delta)
p2 = gmsh.model.geo.addPoint(x0+L, y0+W,z0, delta)
p3 = gmsh.model.geo.addPoint(x0, y0+W, z0, delta)

# Define the perimeter of the rectangle as a loop.
l0 = gmsh.model.geo.addLine(p0, p1)
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p0)
loop = gmsh.model.geo.addCurveLoop([l0,l1,l2,l3])

# Define the interior of the rectangle as a surface.
rectangle = gmsh.model.geo.addPlaneSurface([loop])

# Update the model with all of the features we add.
gmsh.model.geo.synchronize()

# Some geometric objects were only used to define others.
# Identify the physical objects.

# Add each edge separately, so it is a unique object.
# This will allow us to set the boundary conditions separately.
gmsh.model.addPhysicalGroup(1, [l0], 1)
gmsh.model.addPhysicalGroup(1, [l1], 2)
gmsh.model.addPhysicalGroup(1, [l2], 3)
gmsh.model.addPhysicalGroup(1, [l3], 4)

# Define the interior of the rectangle as our domain.
gmsh.model.addPhysicalGroup(2, [rectangle], 1)

# Create a mesh for this system.
gmsh.model.mesh.generate(dim)

# Bring the mesh into FEniCSx.
mesh, subdomains, boundaries = dolfinx.io.gmshio.model_to_mesh(
    gmsh.model, comm=mpi4py.MPI.COMM_WORLD, rank=0, gdim=2)

# Close the mesh generating program.
gmsh.finalize()

In [ ]:
# Get separate identifiers for the four walls.
wall_1 = boundaries.indices[boundaries.values == 1]
wall_2 = boundaries.indices[boundaries.values == 2]
wall_3 = boundaries.indices[boundaries.values == 3]
wall_4 = boundaries.indices[boundaries.values == 4]

In [ ]:
# Plot the entire mesh.
multiphenicsx.io.plot_mesh(mesh)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [ ]:
# Plot the subdomains that FEniCSx has identified.
# There should only be one for this model.
multiphenicsx.io.plot_mesh_tags(subdomains)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [ ]:
# Inspect the boundaries of the elements and the system.
multiphenicsx.io.plot_mesh_tags(boundaries)

# Note that each wall is a different color,
# indicating it is a separate "physical object".

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [ ]:
# Change the last function argument to see the individual walls.
multiphenicsx.io.plot_mesh_entities(mesh, mesh.topology.dim - 1, wall_2)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

# Finite Element Method

Now it is time to tell FEniCSx the problem we want to solve on this mesh.

You can vary the using the cells above geometry above.  The cell below allows you to specify the boundary conditions along the 4 edges.

In the language of finite elements, there are two common types of boundary condtions.
- An ***essential boundary condition*** specifies the value of the function on a boundary.
- A ***natural boundary condtions*** specifies the normal derivative.

The normal derivative is the derivative perpendicular to the boundary.  It is possible to set the function or its normal derivative equal to any other function on the boundary.  These examples will set the function equal to a constant or its normal derivative equal to zero.

In [ ]:
# Set the potential on each wall.
V1 = 2.0
V2 = 2.0
V3 = 0.0
V4 = 0.0

# Define the type of boundary on each wall.
# Set to "True" or "1" for essential (fixed potential).
# Set to "False" or "0" for natural (zero normal derivative).
essential_1 = True
essential_2 = True
essential_3 = True
essential_4 = True

In [ ]:
## Set up the finite element problem.

# Define trial and test functions.
V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 2))

# Define the trial and test functions.
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Create a function to store the solution.
phi = dolfinx.fem.Function(V)

# Identify the domain (all the points inside the boundary).
Omega = subdomains.indices[subdomains.values == 1]

# Identify the boundary for FEniCSx.
dOmega_1 = dolfinx.fem.locate_dofs_topological(V, boundaries.dim, wall_1)
dOmega_2 = dolfinx.fem.locate_dofs_topological(V, boundaries.dim, wall_2)
dOmega_3 = dolfinx.fem.locate_dofs_topological(V, boundaries.dim, wall_3)
dOmega_4 = dolfinx.fem.locate_dofs_topological(V, boundaries.dim, wall_4)

# Now introduce the boundary conditions.
# Store the essential boundary conditions in a list.
essential_bc = []
if essential_1:
    Phi0 = dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(V1))
    essential_bc += [dolfinx.fem.dirichletbc(Phi0, dOmega_1, V)]
if essential_2:
    Phi0 = dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(V2))
    essential_bc += [dolfinx.fem.dirichletbc(Phi0, dOmega_2, V)]
if essential_3:
    Phi0 = dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(V3))
    essential_bc += [dolfinx.fem.dirichletbc(Phi0, dOmega_3, V)]
if essential_4:
    Phi0 = dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(V4))
    essential_bc += [dolfinx.fem.dirichletbc(Phi0, dOmega_4, V)]

# This is the FEM version of the Laplacian.
# It is the left-hand side of Poisson's or Laplace's equation.
a = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx

# This the right-hand side of Poisson's equation.
# We need to create a FEniCSx-friendly version of 0.
Zero = dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(0.0))
L = Zero * v * ufl.dx

# Put it all together for FEniCSx.
problem = dolfinx.fem.petsc.LinearProblem(a, L, essential_bc, u=phi)

# Now, solve it!
problem.solve()

# Tie up some loose ends.
phi.vector.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)

In [22]:
# Plot the solution.
multiphenicsx.io.plot_scalar_field(phi, "Potential", warp_factor=1)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [ ]:
# Define a set of elements for a vector field.
W = dolfinx.fem.VectorFunctionSpace(mesh, ("Lagrange", 2))
E = dolfinx.fem.Function(W)

# Compute the gradient as a symbolic expression, then interpolate it onto the mesh.
expr = dolfinx.fem.Expression(ufl.as_vector((-phi.dx(0), -phi.dx(1))), W.element.interpolation_points())
E.interpolate(expr)

In [23]:
# Use multiphenics to plot the vector field.
multiphenicsx.io.plot_vector_field(E,name="Electric Field", glyph_factor=1e-2)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

# Experiments



## 1. Three Grounded Walls

- Create a square: Set the length and width to be equal.
- Set all boundaries to be essential.
- Set the potentials as follows:
  - $V_1=2.0$
  - $V_2=0.0$
  - $V_3=0.0$
  - $V_4=0.0$

Run the model and FEM cells above.  Explore the output.

- Describe the geometry.
- Describe the potential.
- Describe the electric field.

Try to explain the features you see in the potential and field.

In [ ]:
## Adjust the parameters in this cell.
# Define the shape: length and width of the square (equal)
length = 2
width = 2

In [ ]:
# Create a square.
# Locate the center.
x0 = 0
y0 = 0
z0 = 0

# Give input shorter nicknames.
L = length
W = width

# Locate the corner.
x0 = x0 - L/2
y0 = y0 - W/2

# Tell Gmsh how many dimensions we are using.
dim = 2

# Grid size parameter.  Make it smaller for higher resolution.
delta = 0.1

# Create a new model.
gmsh.initialize()
gmsh.model.add("mesh")

# Define points: corners of the rectangle.
p0 = gmsh.model.geo.addPoint(x0,y0, z0, delta)
p1 = gmsh.model.geo.addPoint(x0+L, y0, z0, delta)
p2 = gmsh.model.geo.addPoint(x0+L, y0+W,z0, delta)
p3 = gmsh.model.geo.addPoint(x0, y0+W, z0, delta)

# Define the perimeter of the rectangle as a loop.
l0 = gmsh.model.geo.addLine(p0, p1)
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p0)
loop = gmsh.model.geo.addCurveLoop([l0,l1,l2,l3])

# Define the interior of the rectangle as a surface.
rectangle = gmsh.model.geo.addPlaneSurface([loop])

# Update the model with all of the features we add.
gmsh.model.geo.synchronize()

# Some geometric objects were only used to define others.
# Identify the physical objects.

# Add each edge separately, so it is a unique object.
# This will allow us to set the boundary conditions separately.
gmsh.model.addPhysicalGroup(1, [l0], 1)
gmsh.model.addPhysicalGroup(1, [l1], 2)
gmsh.model.addPhysicalGroup(1, [l2], 3)
gmsh.model.addPhysicalGroup(1, [l3], 4)

# Define the interior of the rectangle as our domain.
gmsh.model.addPhysicalGroup(2, [rectangle], 1)

# Create a mesh for this system.
gmsh.model.mesh.generate(dim)

# Bring the mesh into FEniCSx.
mesh, subdomains, boundaries = dolfinx.io.gmshio.model_to_mesh(
    gmsh.model, comm=mpi4py.MPI.COMM_WORLD, rank=0, gdim=2)

# Close the mesh generating program.
gmsh.finalize()

In [ ]:
# Get separate identifiers for the four walls.
wall_1 = boundaries.indices[boundaries.values == 1]
wall_2 = boundaries.indices[boundaries.values == 2]
wall_3 = boundaries.indices[boundaries.values == 3]
wall_4 = boundaries.indices[boundaries.values == 4]

In [ ]:
# Plot the entire mesh.
multiphenicsx.io.plot_mesh(mesh)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

I ran these new cells here first to make sure setting width equal to length would make a square without the code getting mad, then I was able to run the other model and FEM cells again with the new length and width, and I found that the potential is way more uniform with the square than with the rectangle. For the vector field, I changed the lagrange # to 1 instead of 2, and that changed my rectangle to a square. However, when I changed the 1 back to a 2 to compare, it stayed a square, so I think maybe my program was just running slow.
Anyway, the E field vectors are still focused at the same two corners, but the are deflected a bit, rather than going straight out. The field vectors appear to all be in the same plane, with a net direction that may be just pointing parallel to the borders of the square.
I don't know how to set the boundaries to essential, so that may be missing.

## 2. Two Grounded Walls

- Create a square: Set the length and width to be equal.
- Set all boundaries to be essential.
- Set the potentials as follows:
  - $V_1=2.0$
  - $V_2=0.0$
  - $V_3=2.0$
  - $V_4=0.0$

This will set the potential on opposite sides of the square equal to 2.0.

***Make a prediction before you run the simulation.***  How do you think the potential will change compared to Experiment 1?  What will be the shape of the potential?

**Prediction: the equal and opposite potentials will cancel each other out. This could make the shape a straight line, rather than curved like a slide** 

Run the model and FEM cells above.  Explore the output.

- Describe the geometry.
- Describe the potential.
- Describe the electric field.

Try to explain the features you see in the potential and field.

Having already set the square W and L in the cells above, I just updated the potentials in the FEM cell and re-ran it with the new definitions.
The potential was completely opposite of what I expected. It creates a steep  parabolic curve that is symmetrical when flipping the plot upside down and to the side too. I'm guessing this means V didn't cancel, but was mirrored on the two sides.
The Electric Field now has vectors on all four corners, not just two. This makes me think that the field is projected from any corners connected to the border that has a potential. It looks like the field points slightly away from the line with a potential (so more around the corner, rather than away from the square)

## 3. Natural Boundary Conditions

- Create a square: Set the length and width to be equal.
- Set boundaries 1, 2, and 4 to be essential.
- Set boundary 3 to be natural.
- Set the potentials as follows:
  - $V_1=2.0$
  - $V_2=0.0$
  - $V_3=0.0$
  - $V_4=0.0$

Instead of fixing the value of the potential on the third wall, we are requiring the normal derivative to vanish.

***Make a prediction before you run the simulation.***  How do you think the potential will change compared to Experiment 1?  What will be the shape of the potential?

**Prediction: I think this should look similar to the potential for experiment 1 because the boundary change is on a border that has no potential**

Run the model and FEM cells above.  Explore the output.

- Describe the geometry.
- Describe the potential.
- Describe the electric field.

Try to explain the features you see in the potential and field.

The potential and vector field both look the same as for experiment 1

## 4. Change the Shape

Repeat Experiments 1, 2, and 3, but change the shape from a square to a rectangle.  Try a variety of shapes: for example, $L = 2W$, $L = 100W$, $W=20L$, etc.

In each case, describe what changes, compared to the square.

**Repeat Exeriment 1** 

Set L=2 and W=100 The potential and field vectors look the same, just wayyy longer

Set L=10 and W=1 this actually showed me a little more clearly the difference when boundaries are natural vs essential. When comparing the potential shape in this to the potential shape in my repeats of experiment 3 (my notes for that are below, but I did them first), the rough shape of the potential is the same when L>>W but the whole "tunnel" was more rounded, whereas in this case with all essential boundaries, its like a harsh slope from one end to the next. There's nothing super notable about the vector fields.

**Repeat Exeriment 2** 

Set L=9 and W=3 this potential has a tunnel shape, but rather than rounded on one end and squared on the other, it is squared on both ends giving it a flat face. This is because there is potential on both sides, which tells me that the potential shape flares outward when there is potential vs when there is not. There are vectors on all 4 corners, but they all point more toward the shorter end of the rectangle, so I'm going to make W>L and see what happens.

Set L=2 and W=5 With potential still on two sides, the shape is still more of a tunnel than a slide, but the tunnel is much taller now because the potential is on the sides that are shorter. Additionally, the E vectors are a little more crazy - they're pointing mostly directly out from the edge with potential on one end, but all over the place on the other end. Because all my boundaries are essential, I was surprised to see this difference!

**Repeat Exeriment 3** 

Set L=2 and W=8 This caused a super stretched out potential, but the vector field looked exaclty the same as with the square, it was just on a rectangle mesh instead of a square mesh

Set L=8 and W=2 I expected this to be exactly the same as L=2 and W=8, but I was very wrong. Most obviously, the potential was still along the same border (on a length side rather than width), so before, the potential vectors were on two corners of a short side of the rectangle, but they're now on two corners of a long side of the rectangle. More interesting, however, was the change in potential. It no longer looks like a slide, but like a really wide tunnel with a rounded opening and a squared openeing. I'ts pretty cool.

Set L=80 and W=2 same shape potential as when L=8 and W=2, but wayyyyy wider. The vector field is harder to view because the rectangle is so long, but it seems like the vectors and therefore E is smaller. Could this be because the line with the potential is longer? 

## 5. Change the Boundary Conditions

Repeat Experiment 1, then explore the effect of changing the boundary conditions.  Change some of the potentials on the four walls.  Change some of the boundary conditions from essential to natural.

***Describe your experiments and your findings below.***

I first set L=W=2 with essential boundaries and V1 = 2.0 to reset the models.

From there, the first thing I messed with was potential; I wanted to see what happened if all four walls had potential, so I set the following:

- $V_1=2.0$
- $V_2=2.0$
- $V_3=2.0$
- $V_4=2.0$

This was pretty cool! It resulted in a completely flat potential shape, but it did have a "hot" spot (yellow) and a "cold" spot (purple) - not sure why! The vectors however, disappeared! Like I had predicted earlier, the presence of potential on all 4 sides canceled out the field - it just needed to be on all 4 sides not just two opposite sides :)

Next, I wanted to see what happened if two adjactent walls had potential, so I set the following:

- $V_1=2.0$
- $V_2=2.0$
- $V_3=0.0$
- $V_4=0.0$

This was my favorite potential shape! It's very hard to explain, so I'm leaving the FEM model set to it. The vectors were really interesting too - there were only vectors on two opposite corners; seemingly, the field on the shared corner between the two potentials canceled out.

I wanted to compare changes in boundary conditions to shapes/potentials I already expected, so I changed boundary conditions, where for each boundary condition I set potentials 1 and 2 equal to 2.0 and potentials 3 and 4 equal to 0. I chose this, rather than all potentials, becauese it wouldn't be very exciting if the vectors were cancelled out each time.

First, because I knew what the potential and vector field looked like when all boundaries were essential, I set all 4 to natural. This gave no potential (flat and all one color). It also had no vector fields, so that didn't go according to plan.

Next, I set the edges with potentials to essential and left the edges withuot potentials as natural. This gave a very small potential (still flat, but it has colors now), but still no vectors... hmmmm

So I swapped the natural and essential vectors. This brought me back to no potential and no E vector field, so the boundaries must be essential where there is a potential (a nice little rhyme)

To test this I set just V1=2 with all other potentials at 0, and set all boundaries to natural except boundary 1. This brought back the colored potential (though still flat), but no vectors, so the other boundaries must have some influence.

Then, all boundaries to essential except boundary 1. This was still boring, and I've been unable to find a situation with natural boundaries (except experiment 3) that had any vectors.

## Challenge: Charge in a Box

In the space below, try to merge commands from this notebook and Notebook 6 to add a charge to the box, compute the potential, and plot the potential and field in the system.

***Replace with your notes.***

In [ ]:
## Replace with your code.

# Reflection and Summary

- What are the major takeaways of this assignment for you?
  - I feel like this helped me understand how potentials interact
- What was the most difficult part of this assignment?
  - Reading through each line of code to figure out what code does what
- What was the most interesting part of this assignment?
  - Seeing the potential shape when two adjacent sides had potential!
- What questions do you have?
  - I'm still wondering the actual difference in natural vs essential boundaries. I understand the technical difference and saw how it affects the potential, but it doesn't really make sense. Are boundaries different from boundary conditions? It's just a difficult concept for me overall

***Replace with your response.***